#Final B4 - Grupo 1 - Preparación del DF para Machine Learning

Análisis de DF de clientes de IPTV para detectar bajas en un período de dos meses aplicando modelos predictivos.

Integrantes:



##**Importación de librerías**

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Carga inicial del DF**

Se carga el archivo generado anteriormente con consultas mensuales a la DB.

In [3]:
# df = pd.read_csv('./sample_data' + '/foto_mes_281122.csv')

path = '/content/drive/My Drive/B4/TPFinal'
df = pd.read_csv(path + '/Datos/foto_mes_281122.csv')


## **Limpieza inicial sobre el csv**

In [4]:
df2=df.copy()
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,DISCRIMINATOR,EstadoCuenta,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,fechaInstalacion,FechaAltaCliente,puntoCentroGoogleMaps_x,puntoCentroGoogleMaps_y,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote
0,2021-05-31 23:59:59,50016,537453,FISICA,Alta,Habilitado,0.00,0,1,2017-04-14 08:53:18.000,1984-08-20 00:00:00.000,"-37,10254858","-56,84453842",0,0,0,0,1,3,5
1,2021-05-31 23:59:59,50016,672504,FISICA,Alta,Habilitado,0.00,0,0,2019-07-16 14:18:24.000,1984-08-20 00:00:00.000,"-37,09822111","-56,86107393",0,1,0,0,0,4,4
2,2021-05-31 23:59:59,50024,433890,FISICA,Alta,Habilitado,0.17,1,0,2015-04-24 12:03:08.000,1987-08-20 00:00:00.000,"-37,140466","-56,88160596",0,1,0,0,0,2,4
3,2021-05-31 23:59:59,50125,455362,FISICA,Alta,Habilitado,0.00,1,0,2015-09-29 10:32:38.000,1985-08-13 00:00:00.000,"-37,10533035","-56,86953262",0,1,0,0,0,1,0
4,2021-05-31 23:59:59,50157,505289,FISICA,Alta,Habilitado,0.00,0,0,2016-12-07 12:47:29.000,1985-09-17 00:00:00.000,"-37,10218354","-56,85299984",0,0,0,0,0,3,0


In [5]:
display(df2.dtypes)


foto_mes                            object
numeroCliente                        int64
ServicioIPTV                         int64
DISCRIMINATOR                       object
EstadoCuenta                        object
EstadoServicio                      object
saldo                              float64
debitoAutomatico                     int64
cantidadReclamos                     int64
fechaInstalacion                    object
FechaAltaCliente                    object
puntoCentroGoogleMaps_x             object
puntoCentroGoogleMaps_y             object
paqueteHBO                           int64
paqueteHD                            int64
paqueteAdultos                       int64
PaqueteFutbol                        int64
TieneBonificacion                    int64
cantidadOtrosServiciosMismoLote      int64
cantidadOtrosServiciosOtroLote       int64
dtype: object

In [6]:
df2 = df2.drop(['puntoCentroGoogleMaps_x', 'puntoCentroGoogleMaps_y'], axis='columns')
print(df2.shape)


(26521, 18)


## **Tratamiento de columnas fecha**

In [7]:
# Se llavan los formatos de fecha a DateTime (se utilizarán después)
df2['FechaActual'] = df2['foto_mes'].astype('datetime64[ns]')
df2['fechaInstalacion'] = df2['fechaInstalacion'].astype('datetime64[ns]')
df2['FechaAltaCliente'] = df2['FechaAltaCliente'].astype('datetime64[ns]')

display(df2.dtypes)


foto_mes                                   object
numeroCliente                               int64
ServicioIPTV                                int64
DISCRIMINATOR                              object
EstadoCuenta                               object
EstadoServicio                             object
saldo                                     float64
debitoAutomatico                            int64
cantidadReclamos                            int64
fechaInstalacion                   datetime64[ns]
FechaAltaCliente                   datetime64[ns]
paqueteHBO                                  int64
paqueteHD                                   int64
paqueteAdultos                              int64
PaqueteFutbol                               int64
TieneBonificacion                           int64
cantidadOtrosServiciosMismoLote             int64
cantidadOtrosServiciosOtroLote              int64
FechaActual                        datetime64[ns]
dtype: object

In [8]:
df2['foto_mes'] = df2['foto_mes'].astype(str).str[:4] + df2['foto_mes'].astype(str).str[5:7]
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,DISCRIMINATOR,EstadoCuenta,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,fechaInstalacion,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual
0,202105,50016,537453,FISICA,Alta,Habilitado,0.00,0,1,2017-04-14 08:53:18,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59
1,202105,50016,672504,FISICA,Alta,Habilitado,0.00,0,0,2019-07-16 14:18:24,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59
2,202105,50024,433890,FISICA,Alta,Habilitado,0.17,1,0,2015-04-24 12:03:08,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59
3,202105,50125,455362,FISICA,Alta,Habilitado,0.00,1,0,2015-09-29 10:32:38,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59
4,202105,50157,505289,FISICA,Alta,Habilitado,0.00,0,0,2016-12-07 12:47:29,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59


In [9]:
df2['foto_mes'] = df2['foto_mes'].astype('int')
display(df2.dtypes)


foto_mes                                    int64
numeroCliente                               int64
ServicioIPTV                                int64
DISCRIMINATOR                              object
EstadoCuenta                               object
EstadoServicio                             object
saldo                                     float64
debitoAutomatico                            int64
cantidadReclamos                            int64
fechaInstalacion                   datetime64[ns]
FechaAltaCliente                   datetime64[ns]
paqueteHBO                                  int64
paqueteHD                                   int64
paqueteAdultos                              int64
PaqueteFutbol                               int64
TieneBonificacion                           int64
cantidadOtrosServiciosMismoLote             int64
cantidadOtrosServiciosOtroLote              int64
FechaActual                        datetime64[ns]
dtype: object

## **Creación de variable objetivo**

In [10]:
# Creo un lag (lo voy a usar para crear la variable objetivo)
df2['Mes_1'] = df2.groupby(['ServicioIPTV'])['foto_mes'].shift(-1)
df2['Mes_2'] = df2.groupby(['ServicioIPTV'])['foto_mes'].shift(-2)

# Despues de crear el lag puedo descartar foto_mes > 202109
df2=df2[df2.foto_mes < 202110]

# Verifico el formato de la tabla
df2[['ServicioIPTV', 'foto_mes', 'Mes_1', 'Mes_2']].head(10)


,ServicioIPTV,foto_mes,Mes_1,Mes_2
0,537453,202105,202106.0,202107.0
1,672504,202105,202106.0,202107.0
2,433890,202105,202106.0,202107.0
3,455362,202105,202106.0,202107.0
4,505289,202105,202106.0,202107.0
5,382780,202105,202106.0,202107.0
6,555178,202105,202106.0,202107.0
7,816076,202105,202106.0,202107.0
8,556933,202105,202106.0,202107.0
9,629008,202105,202106.0,202107.0


In [11]:
df2['target'] = "Continua"
df2.loc[(df2['Mes_2'].isnull()), 'target'] = "Baja+2"
df2.loc[(df2['Mes_1'].isnull()), 'target'] = "Baja+1"


In [12]:
df2['target_bin'] = np.where(df2['target']=='Baja+2', 1, 0)


In [13]:
df2 = df2.drop(['Mes_1' , 'Mes_2', 'target'] , axis='columns')
print(df2.shape)


(19034, 20)


In [15]:
df2.target_bin.value_counts()


0    18482
1      552
Name: target_bin, dtype: int64

Tenemos un dataset fuertemente desbalanceado: 18.482 elementos son de la clase 0 (mayoritaria, que permanece en el servicio) y 552 elementos de la clase 1 (minoritaria, gente que abandona el servicio). La clase 1 es sólo el 2,9% del dataset.

## **Limpieza y transformación del DF**

In [16]:
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,DISCRIMINATOR,EstadoCuenta,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,fechaInstalacion,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual,target_bin
0,202105,50016,537453,FISICA,Alta,Habilitado,0.00,0,1,2017-04-14 08:53:18,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59,0
1,202105,50016,672504,FISICA,Alta,Habilitado,0.00,0,0,2019-07-16 14:18:24,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59,0
2,202105,50024,433890,FISICA,Alta,Habilitado,0.17,1,0,2015-04-24 12:03:08,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59,0
3,202105,50125,455362,FISICA,Alta,Habilitado,0.00,1,0,2015-09-29 10:32:38,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59,0
4,202105,50157,505289,FISICA,Alta,Habilitado,0.00,0,0,2016-12-07 12:47:29,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59,0


In [17]:
# La columna DISCRIMINATOR indica si el cliente es una presona física o jurídica.
# Reemplazamos esta columna por otra más descriptiva PersonaFisica con valores 1 y 0 según corresponda.
df2.insert(3,'PersonaFisica', 1, True)
df2['PersonaFisica'] = np.where(df2['DISCRIMINATOR']=='FISICA', 1, 0)
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,PersonaFisica,DISCRIMINATOR,EstadoCuenta,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,...,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual,target_bin
0,202105,50016,537453,1,FISICA,Alta,Habilitado,0.00,0,1,...,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59,0
1,202105,50016,672504,1,FISICA,Alta,Habilitado,0.00,0,0,...,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59,0
2,202105,50024,433890,1,FISICA,Alta,Habilitado,0.17,1,0,...,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59,0
3,202105,50125,455362,1,FISICA,Alta,Habilitado,0.00,1,0,...,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59,0
4,202105,50157,505289,1,FISICA,Alta,Habilitado,0.00,0,0,...,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59,0


In [18]:
df2 = df2.drop(['DISCRIMINATOR'], axis='columns')
df2.PersonaFisica.value_counts()


1    18120
0      914
Name: PersonaFisica, dtype: int64

In [19]:
# La columna EstadoCuenta indica si el cliente está en una situación normal (Alta) o su cuenta está refinanciada (Refinanciada).
# Reemplazamos esta columna por otra más descriptiva CuentaRefinanciada con valores 1 y 0 según corresponda.
df2.insert(4,'CuentaRefinanciada', 0, True)
df2['CuentaRefinanciada'] = np.where(df2['EstadoCuenta']=='Refinanciada', 1, 0)
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,PersonaFisica,CuentaRefinanciada,EstadoCuenta,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,...,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual,target_bin
0,202105,50016,537453,1,0,Alta,Habilitado,0.00,0,1,...,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59,0
1,202105,50016,672504,1,0,Alta,Habilitado,0.00,0,0,...,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59,0
2,202105,50024,433890,1,0,Alta,Habilitado,0.17,1,0,...,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59,0
3,202105,50125,455362,1,0,Alta,Habilitado,0.00,1,0,...,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59,0
4,202105,50157,505289,1,0,Alta,Habilitado,0.00,0,0,...,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59,0


In [20]:
df2 = df2.drop(['EstadoCuenta'], axis='columns')
df2.CuentaRefinanciada.value_counts()


0    18781
1      253
Name: CuentaRefinanciada, dtype: int64

In [21]:
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,PersonaFisica,CuentaRefinanciada,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,fechaInstalacion,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual,target_bin
0,202105,50016,537453,1,0,Habilitado,0.00,0,1,2017-04-14 08:53:18,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59,0
1,202105,50016,672504,1,0,Habilitado,0.00,0,0,2019-07-16 14:18:24,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59,0
2,202105,50024,433890,1,0,Habilitado,0.17,1,0,2015-04-24 12:03:08,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59,0
3,202105,50125,455362,1,0,Habilitado,0.00,1,0,2015-09-29 10:32:38,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59,0
4,202105,50157,505289,1,0,Habilitado,0.00,0,0,2016-12-07 12:47:29,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59,0


In [22]:
# La columna EstadoServicio indica si el servicio está en una situación normal (Habilitado) o está cortadp preventivamente por falta de pago
# (Corte por falta de pago). Reemplazamos esta columna por otra más descriptiva ServicioHabilitado con valores 1 y 0 según corresponda.
df2.insert(5,'ServicioHabilitado', 1, True)
df2['ServicioHabilitado'] = np.where(df2['EstadoServicio']=='Habilitado', 1, 0)
df2.head()


,foto_mes,numeroCliente,ServicioIPTV,PersonaFisica,CuentaRefinanciada,ServicioHabilitado,EstadoServicio,saldo,debitoAutomatico,cantidadReclamos,...,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual,target_bin
0,202105,50016,537453,1,0,1,Habilitado,0.00,0,1,...,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59,0
1,202105,50016,672504,1,0,1,Habilitado,0.00,0,0,...,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59,0
2,202105,50024,433890,1,0,1,Habilitado,0.17,1,0,...,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59,0
3,202105,50125,455362,1,0,1,Habilitado,0.00,1,0,...,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59,0
4,202105,50157,505289,1,0,1,Habilitado,0.00,0,0,...,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59,0


In [23]:
df2 = df2.drop(['EstadoServicio'], axis='columns')
df2.ServicioHabilitado.value_counts()


1    18910
0      124
Name: ServicioHabilitado, dtype: int64

In [24]:
# La columna saldo indica cuanto debe la persona en ese mes. Se modificarán los NULL con cero.
print(df2['saldo'].isna().sum())

df2.saldo.fillna(0, inplace=True)
print(df2['saldo'].isna().sum())


567
0


In [25]:
# DebitoAutomatico indica si la persona paga con Débito Automático. No se necesitan transformaciones.
# CantidadReclamos es la cantidad de reclamos que se hicieron sobre ese servicio en los últimos 6 meses. No se necesitan transformaciones.
df2.rename(columns = {'numeroCliente':'NumeroCliente', 'saldo':'Saldo', 'debitoAutomatico':'DebitoAutomatico', 'cantidadReclamos':'CantidadReclamos'}, inplace = True)
df2.head()


,foto_mes,NumeroCliente,ServicioIPTV,PersonaFisica,CuentaRefinanciada,ServicioHabilitado,Saldo,DebitoAutomatico,CantidadReclamos,fechaInstalacion,FechaAltaCliente,paqueteHBO,paqueteHD,paqueteAdultos,PaqueteFutbol,TieneBonificacion,cantidadOtrosServiciosMismoLote,cantidadOtrosServiciosOtroLote,FechaActual,target_bin
0,202105,50016,537453,1,0,1,0.00,0,1,2017-04-14 08:53:18,1984-08-20,0,0,0,0,1,3,5,2021-05-31 23:59:59,0
1,202105,50016,672504,1,0,1,0.00,0,0,2019-07-16 14:18:24,1984-08-20,0,1,0,0,0,4,4,2021-05-31 23:59:59,0
2,202105,50024,433890,1,0,1,0.17,1,0,2015-04-24 12:03:08,1987-08-20,0,1,0,0,0,2,4,2021-05-31 23:59:59,0
3,202105,50125,455362,1,0,1,0.00,1,0,2015-09-29 10:32:38,1985-08-13,0,1,0,0,0,1,0,2021-05-31 23:59:59,0
4,202105,50157,505289,1,0,1,0.00,0,0,2016-12-07 12:47:29,1985-09-17,0,0,0,0,0,3,0,2021-05-31 23:59:59,0


In [26]:
# fechaInstalacion es la fecha cuando el servicio fue dado de alta.
# Reemplazamos esta columna por MesesServicio indicando la cantidad de meses que está en operación el servicio
df2.insert(9,'MesesServicio', 0, True)
df2['MesesServicio'] = ((df2.FechaActual - df2.fechaInstalacion)/np.timedelta64(1, 'M'))


In [27]:
# FechaAltaCliente es la fecha cuando el cliente fue dado de alta.
# Reemplazamos esta columna por MesesCliente indicando la cantidad de meses que está el cliente como abonado (antigüedad)
df2.insert(10,'MesesCliente', 0, True)
df2['MesesCliente'] = ((df2.FechaActual - df2.FechaAltaCliente)/np.timedelta64(1, 'M'))


In [28]:
df2 = df2.drop(['fechaInstalacion', 'FechaAltaCliente', 'FechaActual'], axis='columns')
display(df2.dtypes)


foto_mes                             int64
NumeroCliente                        int64
ServicioIPTV                         int64
PersonaFisica                        int64
CuentaRefinanciada                   int64
ServicioHabilitado                   int64
Saldo                              float64
DebitoAutomatico                     int64
CantidadReclamos                     int64
MesesServicio                      float64
MesesCliente                       float64
paqueteHBO                           int64
paqueteHD                            int64
paqueteAdultos                       int64
PaqueteFutbol                        int64
TieneBonificacion                    int64
cantidadOtrosServiciosMismoLote      int64
cantidadOtrosServiciosOtroLote       int64
target_bin                           int64
dtype: object

In [29]:
df2.rename(columns = {'paqueteHBO':'PaqueteHBO', 'paqueteHD':'PaqueteHD', 'paqueteAdultos':'PaqueteAdultos'}, inplace = True)
df2.rename(columns = {'cantidadOtrosServiciosMismoLote':'CantOtrosServMismoLote', 'cantidadOtrosServiciosOtroLote':'CantOtrosServOtroLote'}, inplace = True)
df2.head()


,foto_mes,NumeroCliente,ServicioIPTV,PersonaFisica,CuentaRefinanciada,ServicioHabilitado,Saldo,DebitoAutomatico,CantidadReclamos,MesesServicio,MesesCliente,PaqueteHBO,PaqueteHD,PaqueteAdultos,PaqueteFutbol,TieneBonificacion,CantOtrosServMismoLote,CantOtrosServOtroLote,target_bin
0,202105,50016,537453,1,0,1,0.00,0,1,49.565852,441.372512,0,0,0,0,1,3,5,0
1,202105,50016,672504,1,0,1,0.00,0,0,22.518865,441.372512,0,1,0,0,0,4,4,0
2,202105,50024,433890,1,0,1,0.17,1,0,73.249892,405.396414,0,1,0,0,0,2,4,0
3,202105,50125,455362,1,0,1,0.00,1,0,68.060885,429.610464,0,1,0,0,0,1,0,0
4,202105,50157,505289,1,0,1,0.00,0,0,53.765934,428.460543,0,0,0,0,0,3,0,0


In [30]:
df2[['MesesServicio', 'MesesCliente']] = df2[['MesesServicio', 'MesesCliente']].round(2)
df2.head()


,foto_mes,NumeroCliente,ServicioIPTV,PersonaFisica,CuentaRefinanciada,ServicioHabilitado,Saldo,DebitoAutomatico,CantidadReclamos,MesesServicio,MesesCliente,PaqueteHBO,PaqueteHD,PaqueteAdultos,PaqueteFutbol,TieneBonificacion,CantOtrosServMismoLote,CantOtrosServOtroLote,target_bin
0,202105,50016,537453,1,0,1,0.00,0,1,49.57,441.37,0,0,0,0,1,3,5,0
1,202105,50016,672504,1,0,1,0.00,0,0,22.52,441.37,0,1,0,0,0,4,4,0
2,202105,50024,433890,1,0,1,0.17,1,0,73.25,405.40,0,1,0,0,0,2,4,0
3,202105,50125,455362,1,0,1,0.00,1,0,68.06,429.61,0,1,0,0,0,1,0,0
4,202105,50157,505289,1,0,1,0.00,0,0,53.77,428.46,0,0,0,0,0,3,0,0


In [31]:
# Me fijo si quedó algún NaN
df2.isnull().any()


foto_mes                  False
NumeroCliente             False
ServicioIPTV              False
PersonaFisica             False
CuentaRefinanciada        False
ServicioHabilitado        False
Saldo                     False
DebitoAutomatico          False
CantidadReclamos          False
MesesServicio              True
MesesCliente              False
PaqueteHBO                False
PaqueteHD                 False
PaqueteAdultos            False
PaqueteFutbol             False
TieneBonificacion         False
CantOtrosServMismoLote    False
CantOtrosServOtroLote     False
target_bin                False
dtype: bool

In [32]:
# Veo cuantos registros NULL tengo en MesesServicio
print(df2['MesesServicio'].isna().sum())


200


In [33]:
# Reemplazamos los valores faltantes con la media de la columna (son relativamente pocos los valores faltantes)
promedio = df2['MesesServicio'].mean() 
df2['MesesServicio'].fillna(value=promedio, inplace=True)

df2.isnull().any()


foto_mes                  False
NumeroCliente             False
ServicioIPTV              False
PersonaFisica             False
CuentaRefinanciada        False
ServicioHabilitado        False
Saldo                     False
DebitoAutomatico          False
CantidadReclamos          False
MesesServicio             False
MesesCliente              False
PaqueteHBO                False
PaqueteHD                 False
PaqueteAdultos            False
PaqueteFutbol             False
TieneBonificacion         False
CantOtrosServMismoLote    False
CantOtrosServOtroLote     False
target_bin                False
dtype: bool

**Conclusión Limpieza DF**

Luego de limpiar el DF se lo guarda para usarlo posteriormente en otros notebooks (estudios de Machine Learning). Guardo todos los meses Mayo (202105) a Setiembre (202108). Después separar en Train y Test.

In [ ]:
# Guardamos el DF total
# df2.to_csv('./sample_data' + '/DFTotal.csv', index=False)
df2.to_csv(path + '/ML/DFTotal.csv', index=False)
